# Продвинутый Python, лекция 11

**Лектор:** Петров Тимур

**Семинаристы:** Петров Тимур, Коган Александра, Романченко Полина

**Spoiler Alert:** в рамках курса нельзя изучить ни одну из тем от и до досконально (к сожалению, на это требуется больше времени, чем даже 3 часа в неделю). Но мы попробуем рассказать столько, сколько возможно :)

Так как мы проходим веб-разработку, то это явно не та вещь, которая делается через colab. Поэтому здесь написан просто код, который можно воспроизвести локально

## Деплоим страницу с приложением!

Поскольку в нашем текущем мире вариантов осталось не так много, то будем использовать Ya.Cloud (он платный, но есть пробный период с кэшем в размере 4к рублей, нам подойдет)

Используем ВМ для [Django](https://cloud.yandex.ru/marketplace/products/yc/django-3), создаем все, что необходимо (указываем ключи etc):

```
ssh-keygen -t ed25519
```

Вставляем и наслаждаемся жизнью! Теперь коннект делается через:

```
ssh -i <your private key> <name>@<public port>
```

(Для удобства можно подключиться внутри PyCharm)

И все, дальше внутри можно делать свой собственный проект внутри)

## Dash

Говоря про веб-разработку нельзя забывать, что очень часто веб-страницы вам нужны для аналитики, чтобы показать всем красивые дашборды etc. Так как это очень частый запрос, то для этого есть отдельные бибилиотеки для визуализации. Мы разберем 2 бибилотеки: Dash и StreamLit

Что такое Dash? Напомним эволюцию развития визализаций:

* Matplotlib - база

* Seaborn - скомпоновали более красиво и просто

* Plotly - добавили красивую прорисовку и JS

* Dash - добавили в Plotly еще и Flask!

То есть мы с вами знаем обе части Dash, так что понимание, как оно работает, не должно составить труда

In [ ]:
!pip install dash
!pip install jupyter-dash

In [ ]:
from dash import Dash, html, dcc
import plotly.express as px
import pandas as pd

app = Dash(__name__) # Строим дашик (знакомый синтаксис, не правда ли?)

colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}


df = pd.DataFrame({
    "Fruit": ["Apples", "Oranges", "Bananas", "Apples", "Oranges", "Bananas"],
    "Amount": [4, 1, 2, 2, 4, 5],
    "City": ["SF", "SF", "SF", "Montreal", "Montreal", "Montreal"]
})


fig = px.bar(df, x="Fruit", y="Amount", color="City", barmode="group") #опа, Plotly

fig.update_layout(
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    font_color=colors['text']
)

app.layout = html.Div(children=[
    html.H1(children='Hello Dash', style={'textAlign': 'center', 'color': '#7FDBFF'}),

    html.Div(children='''
        Dash: A web application framework for your data.
    '''),

    dcc.Graph(
        id='example-graph',
        figure=fig
    )
])

if __name__ == '__main__':
    app.run_server(debug=True) #запускаем сервер

Что здесь видим?

Часть из Plotly, связанной с отрисовкой графиков

Часть из Flask, свзяанной с запуском приложения

Но есть еще одна часть - [html](https://dash.plotly.com/dash-html-components) и [dcc](https://dash.plotly.com/dash-core-components) (эта часть отвечает за то, чтобы сделать наш добрый HTML-документ)

В данном случае html - это часть, которая имеет все необходимые тэги 

Внутри элементов HTML есть различные аргументы (которых достаточно много отдельных, тут лушче читать документацию), но основные, которые есть везде:

* children - что находится внутри (то есть отвечает за вложенность структуры)

* style - стиль (передаем как будто css-ка)

* id - параметр, нужный для callback

DCC - часть, которая необходима для отображения Plotly частей (будь то слайдер, график, выбор вариантов etc)

### Callback

Мы явно хотим иметь вские поля для воода, селекторы etc и их каким-то образом отрабатывать, что необходимо отдельно прописывать. Это делается с помощью callback (по существу, это функции, которые будут отрабаывать изменения через наши POST-запросы)

Давайте попробуем такой написать:

In [ ]:
from dash import Dash, dcc, html, Input, Output

## Часть с самим отрисовыванием

app = Dash(__name__)

app.layout = html.Div([
    html.H6("Change the value in the text box to see callbacks in action!"),
    html.Div([
        "Input: ",
        dcc.Input(id='my-input', value='initial value', type='text')
    ]),
    html.Br(),
    html.Div(id='my-output'),

])

## Наш Callback

@app.callback(
    Output(component_id='my-output', component_property='children'), ## Куда выводить
    Input(component_id='my-input', component_property='value') ## Откуда брать
)
def update_output_div(input_value):
    return f'Output: {input_value}' ## Мы вовзращаем некоторую строку, которая будет вставлена в компонент с id = my-output в children


if __name__ == '__main__':
    app.run_server(debug=True)

Но в данном случае все достаточно просто. Мы никаким образом не меняли ничего в данных etc. Давайте попробуем сделать все то же самое, но только уже с данными:

In [ ]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminderDataFiveYear.csv')

app = Dash(__name__)

app.layout = html.Div([
    dcc.Graph(id='graph-with-slider'),
    dcc.Slider(
        df['year'].min(),
        df['year'].max(),
        step=None,
        value=df['year'].min(),
        marks={str(year): str(year) for year in df['year'].unique()},
        id='year-slider'
    )
])


@app.callback(
    Output('graph-with-slider', 'figure'), # говорим, что меняем фигуру (то есть прорисовываем ее заново)
    Input('year-slider', 'value'))
def update_figure(selected_year):
    filtered_df = df[df.year == selected_year]

    fig = px.scatter(filtered_df, x="gdpPercap", y="lifeExp",
                     size="pop", color="continent", hover_name="country",
                     log_x=True, size_max=55)

    fig.update_layout(transition_duration=500)
    return fig


if __name__ == '__main__':
    app.run_server(debug=True)

Если у нас несколько селекторов, то надо прописывать несколько input на каждое значение. Логика простая:

* есть данные, нам их надо поменять

* забираем изменения через Input (аргументы идут в порядке вызова Input и меняем. Аналогично с Output - перечисляем и возвращаем в том же порядке, в котором давали Output)

Как можно понять, здесь уже можно спокойно делать несколько разных фильтров на один и тот же график)

### Callback от других графиков

А теперь допустим, что я хочу что-то выбрать на одном графике и чтобы остальные графики обновлялись при этом (допустим, тыкаем на точку, чтобы получить информацию на других графиках)

Тоже можно!

In [ ]:
from dash import Dash, html, dcc, Input, Output
import pandas as pd
import plotly.express as px

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=external_stylesheets)

df = pd.read_csv('https://plotly.github.io/datasets/country_indicators.csv')


app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                df['Indicator Name'].unique(),
                'Fertility rate, total (births per woman)',
                id='crossfilter-xaxis-column',
            ),
            dcc.RadioItems(
                ['Linear', 'Log'],
                'Linear',
                id='crossfilter-xaxis-type',
                labelStyle={'display': 'inline-block', 'marginTop': '5px'}
            )
        ],
        style={'width': '49%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                df['Indicator Name'].unique(),
                'Life expectancy at birth, total (years)',
                id='crossfilter-yaxis-column'
            ),
            dcc.RadioItems(
                ['Linear', 'Log'],
                'Linear',
                id='crossfilter-yaxis-type',
                labelStyle={'display': 'inline-block', 'marginTop': '5px'}
            )
        ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
    ], style={
        'padding': '10px 5px'
    }),

    html.Div([
        dcc.Graph(
            id='crossfilter-indicator-scatter',
            hoverData={'points': [{'customdata': 'Japan'}]}
        )
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    html.Div([
        dcc.Graph(id='x-time-series'),
        dcc.Graph(id='y-time-series'),
    ], style={'display': 'inline-block', 'width': '49%'}),

    html.Div(dcc.Slider(
        df['Year'].min(),
        df['Year'].max(),
        step=None,
        id='crossfilter-year--slider',
        value=df['Year'].max(),
        marks={str(year): str(year) for year in df['Year'].unique()}
    ), style={'width': '49%', 'padding': '0px 20px 20px 20px'})
])


@app.callback(
    Output('crossfilter-indicator-scatter', 'figure'),
    Input('crossfilter-xaxis-column', 'value'),
    Input('crossfilter-yaxis-column', 'value'),
    Input('crossfilter-xaxis-type', 'value'),
    Input('crossfilter-yaxis-type', 'value'),
    Input('crossfilter-year--slider', 'value'))
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 year_value):
    dff = df[df['Year'] == year_value]

    fig = px.scatter(x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
            y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
            hover_name=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name']
            )

    fig.update_traces(customdata=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'])

    fig.update_xaxes(title=xaxis_column_name, type='linear' if xaxis_type == 'Linear' else 'log')

    fig.update_yaxes(title=yaxis_column_name, type='linear' if yaxis_type == 'Linear' else 'log')

    fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')

    return fig


def create_time_series(dff, axis_type, title):

    fig = px.scatter(dff, x='Year', y='Value')

    fig.update_traces(mode='lines+markers')

    fig.update_xaxes(showgrid=False)

    fig.update_yaxes(type='linear' if axis_type == 'Linear' else 'log')

    fig.add_annotation(x=0, y=0.85, xanchor='left', yanchor='bottom',
                       xref='paper', yref='paper', showarrow=False, align='left',
                       text=title)

    fig.update_layout(height=225, margin={'l': 20, 'b': 30, 'r': 10, 't': 10})

    return fig


@app.callback(
    Output('x-time-series', 'figure'),
    Input('crossfilter-indicator-scatter', 'hoverData'),
    Input('crossfilter-xaxis-column', 'value'),
    Input('crossfilter-xaxis-type', 'value'))
def update_y_timeseries(hoverData, xaxis_column_name, axis_type):
    country_name = hoverData['points'][0]['customdata']
    dff = df[df['Country Name'] == country_name]
    dff = dff[dff['Indicator Name'] == xaxis_column_name]
    title = '<b>{}</b><br>{}'.format(country_name, xaxis_column_name)
    return create_time_series(dff, axis_type, title)


@app.callback(
    Output('y-time-series', 'figure'),
    Input('crossfilter-indicator-scatter', 'hoverData'),
    Input('crossfilter-yaxis-column', 'value'),
    Input('crossfilter-yaxis-type', 'value'))
def update_x_timeseries(hoverData, yaxis_column_name, axis_type):
    dff = df[df['Country Name'] == hoverData['points'][0]['customdata']]
    dff = dff[dff['Indicator Name'] == yaxis_column_name]
    return create_time_series(dff, axis_type, yaxis_column_name)


if __name__ == '__main__':
    app.run_server(debug=True)

## StreamLit

Но можно сделать шаг вперед и отойти от Plotly и Dash (почему? Потому что как минимум полная мощность Dash является платной, money-money) и перейти к более мощным и SOTA инструментам, например, [StreamLit](https://streamlit.io/)

StreamLit - это уже open-source бибилотека, которая работает примерно со всем и подается под соусом "вам вообще веб-разработку знать не надо, все готово из-под коробки"

In [ ]:
!pip install streamlit
!streamlit hello

По базовому демо можете видеть, что уже внутри Streamlit зашито и насколько данный инструмент мощный 🦾

### Графики

Изобразим некоторое число основных графиков, которые можно сделать:

* write() - уникальный инструмент в жанре "нарисуй вот это". Принимает в себя примерно все, что угодно (но он сам решает как и что нарисовать, поэтому не всегда вариант)

* dataframe() - датафрейм

* table() - статичная таблица

* line_chart() - линия

* map() - карта

In [ ]:
import streamlit as st
import numpy as np
import pandas as pd

dataframe = np.random.randn(10, 20)

st.dataframe(dataframe)

dataframe = pd.DataFrame(
    np.random.randn(10, 20),
    columns=('col %d' % i for i in range(20)))

st.dataframe(dataframe.style.highlight_max(axis=0))

dataframe = pd.DataFrame(
    np.random.randn(10, 20),
    columns=('col %d' % i for i in range(20)))
st.table(dataframe)

chart_data = pd.DataFrame(
     np.random.randn(20, 3),
     columns=['a', 'b', 'c'])

st.line_chart(chart_data)

map_data = pd.DataFrame(
    np.random.randn(1000, 2) / [50, 50] + [37.76, -122.4],
    columns=['lat', 'lon'])

st.map(map_data)

### Виджеты

Какие есть виджеты?

* slider() - слайдеры

* text_input() - ввод текста

* checkbox() - галочка

* selectbox() - выбор варианта

* radio() - выбор варианта (но уже другого вида)

In [ ]:
import streamlit as st
import numpy as np
import pandas as pd

x = st.slider('x')
st.write(x, 'squared is', x * x)

st.text_input("Your name", key="name")

if st.checkbox('Show dataframe'):
    chart_data = pd.DataFrame(
       np.random.randn(20, 3),
       columns=['a', 'b', 'c'])
    st.write(chart_data)

#st.session_state.name - можно обратиться к выбору

df = pd.DataFrame({
    'first column': [1, 2, 3, 4],
    'second column': [10, 20, 30, 40]
    })

option = st.selectbox(
    'Which number do you like best?',
     df['first column'])

### Layout

В демо мы видели часть слева. Это называется sidebar(), в который можно выносить все, что вам необходимо:

In [ ]:
# Add a selectbox to the sidebar:
add_selectbox = st.sidebar.selectbox(
    'How would you like to be contacted?',
    ('Email', 'Home phone', 'Mobile phone')
)

# Add a slider to the sidebar:
add_slider = st.sidebar.slider(
    'Select a range of values',
    0.0, 100.0, (25.0, 75.0)
)

Также мы же не всегда хотим тупую линеное отображение, иногда хотим побить на колонки, а это можно сделать вот так с помощью ```columns()```:

In [ ]:
left_column, right_column = st.columns(2)
# You can use a column just like st.sidebar:
left_column.button('Press me!')

# Or even better, call Streamlit functions inside a "with" block:
with right_column:
    chosen = st.radio(
        'Sorting hat',
        ("Gryffindor", "Ravenclaw", "Hufflepuff", "Slytherin"))
    st.write(f"You are in {chosen} house!")

### Странички!

Внутри одного приложения можно даже сделать несколько страничек!

Для этого надо создать отдельно папку pages и прописать новые страницы (таким образом они будут отображаться в sidebar, названия идут по названию скриптов страницы)

## Попугай дня

![](https://upload.wikimedia.org/wikipedia/commons/f/f6/Naturalis_Biodiversity_Center_-_ZMA.AVES.3159_-_Conuropsis_carolinensis_Linnaeus%2C_1758_-_Psittacidae_-_skin_specimen.jpeg)

Это каролингский попугай, почти единственный попугай-эндемик США, к сожалению, вымерший в первую половину XX века. Возможно единственный ядовитый попугай (потому что при поедании его трупа все крысы и прочие погибали, связано в тем, что этот попугай ел ядовитый дурнишник)

В дикой природе он считается вымершим к началу XX века, при этом полностью вымер он только в 1918 году (в зоопарке Цинциннати оставались 2 попугая и погибли, скорее всего, из-за слишком холодной зимы).

Последнего попугая звали Инкас, причем он умер примерно в то же время, что и Марта (последний странствующий голубь). Что более всего странно, что поскольку это был последний погибший попугай, то его тело должно было быть заморожено в Смитсоновском музее. Но он потерялся и никто вообще не знает, где его тело

И вместе с Мартой эти птицы стали одним из символов угрозы вымирания животных, находящихся вокруг нас